In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

### 1. Read in datasets

In [ ]:
codes_mapping = pd.read_excel('srds_customer_identity_adjustment_20220509.xlsx', sheet_name = 'created_proxy_customer_id')

df_0 = pd.read_excel('SRDS SALES BY CUSTOMER AND BY DAY Jan 2020 - Jan 2022.xlsx', skiprows = 4, skipfooter = 5, header = 0)

df_1 = pd.read_excel('SALES BY CUSTOMER AND BY DAY Feb 2022 - July 2022.xlsx', skiprows = 4, skipfooter = 5, header = 0)

df_2 = pd.read_excel('Customers Data by31 Jul 2022.xlsx')

df_3 = pd.read_excel('SRDS Partnership AUGUST TO SEPTEMBER 2022.xlsx', sheet_name = 'SALES DATA AUGUST TO SEPTEMBER', skipfooter = 7)

df_4 = pd.read_excel('S.R.D.S+Ltd_Sales+by+Customer+Summary (6).xlsx', skiprows = 4, skipfooter = 5)

df_5 = pd.read_excel('S.R.D.S+Ltd_Sales+by+Customer+Summary (7).xlsx', skiprows = 4, skipfooter = 5)

df_6 = pd.read_excel('S.R.D.S+Ltd_Sales+by+Customer+Summary (10).xlsx', skiprows = 4, skipfooter = 5)

df_7 = pd.read_excel('S.R.D.S+Ltd_Sales+by+Customer+Summary NOVEMBER 2022.xlsx', skiprows = 4, skipfooter = 5)

df_8 = pd.read_excel('S.R.D.S+Ltd_Sales+by+Customer+Summary December 2022.xlsx', skiprows = 4, skipfooter = 5)

df_9 = pd.read_excel('S.R.D.S Sales by Customer Siummary - January 2023.xlsx', skiprows = 4, skipfooter = 5)

### 2. Cleaning datasets.

In [ ]:
cols_to_str = ["original_customer_id","proxy_customer_id"]
codes_mapping[cols_to_str] = codes_mapping[cols_to_str].astype("str")

In [ ]:
def clean_first_dataset(df):
    
    df.drop(columns = ["Total","SUCCESS"], inplace = True)
    
    #rename Unnamed: O column
    df.rename(columns = {"Unnamed: 0":"vendors"}, inplace = True)
    
    return df

In [ ]:
def clean_second_dataset(df):
    df.drop(columns = ["Total", "Unnamed: 1", "Unnamed: 2"], inplace = True)
    
    df.rename(columns = {"Unnamed: 0":"vendors"}, inplace = True)
    
    return df

In [ ]:
def clean_fourth_dataset(df):
    df.drop(['Total','Unnamed: 2','Unnamed: 3'], axis = 'columns', inplace = True)
    
    return df

In [ ]:
def clean_standard_dataset(df):
    df.drop('Total', axis = 'columns', inplace = True)
    
    return df

In [ ]:
## Applying cleaning functions

df_0_cleaned = clean_first_dataset(df_0)

df_1_cleaned = clean_second_dataset(df_1)

df_2_cleaned = clean_standard_dataset(df_2)

df_3_cleaned = clean_standard_dataset(df_3)

df_4_cleaned = clean_fourth_dataset(df_4)

df_5_cleaned = clean_standard_dataset(df_5)

df_6_cleaned = clean_standard_dataset(df_6)

df_7_cleaned = clean_standard_dataset(df_7)

df_8_cleaned = clean_standard_dataset(df_8)

df_9_cleaned = clean_standard_dataset(df_9)

### 3. Melting dataframes

In [ ]:
def melt_first_two_dfs(df, codes):
    
    """Melt dataframe."""
    
    df_melted = pd.melt(df, id_vars = 'vendors', var_name  = 'date_time', value_name = 'invoice_value')
    
    df_melted['vendors'] = df_melted['vendors'].str.strip()

    df_melted['vendors'] = [x.replace('/ ', '') for x in df_melted['vendors']]
    
    
    df_melted_final = pd.merge(codes_mapping, df_melted, on = 'vendors')
    
    ids = df_melted_final['proxy_customer_id']

    df_melted_final.drop(['original_customer_id','proxy_customer_id'], axis = 'columns', inplace = True)

    df_melted_final.insert(0, 'customer_id', ids)

    df_melted_final['customer_id'] = df_melted_final['customer_id'].astype('int64')
    
    return df_melted_final

In [ ]:
def melt_standard_dfs(df, codes):
    
    """Melt dataframe."""
    
    df_melted = pd.melt(df, id_vars = ['customer_id','vendors'], var_name  = 'date_time', value_name = 'invoice_value')
    
    df_melted['vendors'] = df_melted['vendors'].str.strip()

    df_melted['vendors'] = [x.replace('/ ', '') for x in df_melted['vendors']]
    
    df_melted['customer_id'] = df_melted['customer_id'].astype('int64')
    
    return df_melted

In [ ]:
## Applying melting functions

df_0_melted = melt_first_two_dfs(df_0_cleaned, codes_mapping)

df_1_melted = melt_first_two_dfs(df_1_cleaned, codes_mapping)

df_2_melted = melt_standard_dfs(df_2_cleaned, codes_mapping)

df_3_melted = melt_standard_dfs(df_3_cleaned, codes_mapping)

df_4_melted = melt_standard_dfs(df_4_cleaned, codes_mapping)

df_5_melted = melt_standard_dfs(df_5_cleaned, codes_mapping)

df_6_melted = melt_standard_dfs(df_6_cleaned, codes_mapping)

df_7_melted = melt_standard_dfs(df_7_cleaned, codes_mapping)

df_8_melted = melt_standard_dfs(df_8_cleaned, codes_mapping)

df_9_melted = melt_standard_dfs(df_9_cleaned, codes_mapping)

In [ ]:
## concat all dataframes

final_df = pd.concat([df_0_melted,df_1_melted, df_2_melted, df_3_melted, df_4_melted, df_5_melted, 
                      df_6_melted, df_7_melted, df_8_melted, df_9_melted])

final_df.head(2)

In [ ]:
## As data is shared with overlapping dates, drop duplicates

final_df.drop_duplicates(keep = 'last', inplace = True)

In [ ]:
final_df.shape

In [ ]:
## convert date_time column to datetime and drop any null values

final_df['date_time'] = [datetime.strptime(x, '%d %b, %Y') for x in final_df['date_time']]
    
final_df.dropna(axis = 0, subset = ["invoice_value"], inplace = True)

In [ ]:
import os

os.getcwd()

In [ ]:
final_df.to_csv('/home/jupyter-fiona.kamau/SRDS/rw_srds_bralirwa/data/interim/cleaned_analysis_data.csv', index = False)

In [ ]:
## dvc to track the files